# Demo workflow

Demonstrate simple building of behavior classifier and other analyses

In [1]:
import os
#More reliable to not use GPU here. It's only doing inference with a small net, doesn't take long
os.environ["CUDA_VISIBLE_DEVICES"] = ''

from ethome import create_dataset, interpolate_lowconf_points
from ethome.io import get_sample_data_paths
from ethome.unsupervised import compute_umap_embedding
from ethome.plot import plot_embedding

#Obviously, need to have xgboost installed for this to work...
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict, GroupKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

/home/blansdel/.conda/envs/ethome/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Gather some sample DLC and BORIS tracking and annotation files to play with

In [2]:
tracking_files, boris_files = get_sample_data_paths()

Setup some parameters

In [3]:
N_UMAP_ROWS = 10000

frame_width = 20                 # (float) length of entire horizontal shot
frame_width_units = 'in'         # (str) units frame_width is given in
fps = 30                         # (int) frames per second
resolution = (1200, 1600)        # (tuple) HxW in pixels

animal_renamer = {'adult': 'resident', 'juvenile':'intruder'}

In [5]:
#%% Create dataset and add features
X = create_dataset(tracking_files, 
                         animal_renamer=animal_renamer,
                         labels = boris_files, 
                         frame_width = frame_width, 
                         fps = fps, 
                         frame_width_units = frame_width_units, 
                         resolution = resolution)

All necessary fields provided -- rescaling to 'mm'


In [7]:
X

,resident_x_nose,resident_x_leftear,resident_x_rightear,resident_x_neck,resident_x_lefthip,resident_x_righthip,resident_x_tail,resident_y_nose,resident_y_leftear,resident_y_rightear,...,likelihood_intruder_leftear,likelihood_intruder_rightear,likelihood_intruder_neck,likelihood_intruder_lefthip,likelihood_intruder_righthip,likelihood_intruder_tail,filename,frame,time,label_interact
0,356.207060,352.047492,345.893708,344.032840,320.449575,331.631290,316.902783,100.559235,87.782083,97.712213,...,0.99996,1.00000,0.99989,0.99967,0.99979,1.00,/home/blansdel/projects/ethome/ethome/data/dlc...,0,0.000000,0.0
1,356.207060,352.047492,345.893708,344.032840,320.684843,331.631290,316.902783,102.094030,87.810023,98.218307,...,0.99989,1.00000,0.99936,0.99874,0.99997,1.00,/home/blansdel/projects/ethome/ethome/data/dlc...,1,0.033333,0.0
2,356.207060,352.047492,345.893708,344.032840,320.717863,331.729080,316.902783,102.978268,88.059260,98.226245,...,0.99990,1.00000,0.99964,0.99975,1.00000,1.00,/home/blansdel/projects/ethome/ethome/data/dlc...,2,0.066667,0.0
3,354.648452,351.564257,344.890407,342.207850,320.717863,331.631290,316.833885,103.497380,88.676480,98.573590,...,0.99975,0.99999,0.99734,0.99768,0.99991,1.00,/home/blansdel/projects/ethome/ethome/data/dlc...,3,0.100000,0.0
4,353.351783,351.551875,343.698830,341.871935,320.787078,331.494130,316.788800,103.846947,88.676480,99.542282,...,0.99992,1.00000,0.99939,0.99784,0.99999,1.00,/home/blansdel/projects/ethome/ethome/data/dlc...,4,0.133333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45744,271.894617,275.318220,263.507538,269.503843,286.320865,265.591925,277.518495,289.183763,279.999440,278.909463,...,0.99664,0.99998,0.99415,0.89417,0.94191,0.01,/home/blansdel/projects/ethome/ethome/data/dlc...,10344,344.800000,1.0
45745,271.349787,275.318220,263.507538,268.885353,285.166117,262.692515,274.818157,290.401375,280.829385,280.445845,...,0.99522,0.99993,0.99409,0.96916,0.76056,0.01,/home/blansdel/projects/ethome/ethome/data/dlc...,10345,344.833333,1.0
45746,271.205643,275.318220,263.422765,268.815502,283.640213,261.254875,274.712747,292.032055,282.209558,283.198570,...,0.99951,0.99998,0.99767,0.97600,0.65451,0.01,/home/blansdel/projects/ethome/ethome/data/dlc...,10346,344.866667,0.0
45747,270.222027,275.317903,263.138602,268.583728,283.373513,260.392862,273.791045,292.032055,282.209558,283.198570,...,0.99961,0.99995,0.99666,0.88960,0.75704,0.01,/home/blansdel/projects/ethome/ethome/data/dlc...,10347,344.900000,0.0


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import ipywidgets as widgets

# Helper function to extract x and y coordinates for each animal and body part
def extract_coordinates(frame, prefix):
    x = frame[f"{prefix}_x"]
    y = frame[f"{prefix}_y"]
    return x, y

# Function to update the plot for each frame
def update(frame):
    ax.clear()
    for prefix in prefixes:
        x, y = extract_coordinates(X.iloc[frame], prefix)
        ax.plot(x, y, 'o', label=prefix)
    ax.legend()
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_title(f"Frame {frame}")

# Get unique prefixes (animalid_bodypartid)
prefixes = set(col.rsplit('_', 1)[0] for col in X.columns)

# Find min and max values for x and y for consistent axes
x_min = X.filter(regex='_x$').min().min()
x_max = X.filter(regex='_x$').max().max()
y_min = X.filter(regex='_y$').min().min()
y_max = X.filter(regex='_y$').max().max()

# Create the plot
fig, ax = plt.subplots()
animation = FuncAnimation(fig, update, frames=len(X), interval=100, repeat=True)
plt.close()

# Display the animation as an interactive widget
play = widgets.Play(value=0, min=0, max=len(X)-1, step=1, interval=100, description="Press play", disabled=False)
slider = widgets.IntSlider(min=0, max=len(X)-1, step=1, continuous_update=False)
widgets.jslink((play, 'value'), (slider, 'value'))
widgets.VBox([widgets.HBox([play, slider]), HTML(animation.to_jshtml())])

Smooth over low-confidence points

In [ ]:
interpolate_lowconf_points(dataset)

Now create features on this dataset. Can use pre-built featuresets, or make your own. 
Here are two that work with a mouse resident-intruder setup:

In [ ]:
dataset.features.add('cnn1d_prob')
dataset.features.add('mars')

## Supervised learning

Now we have features, we can train a behavior classifier

In [ ]:
splitter = GroupKFold(n_splits = dataset.metadata.n_videos)

print("Fitting ML model with (group) LOO CV")
predictions = cross_val_predict(XGBClassifier(), 
                                dataset.ml.features, 
                                dataset.ml.labels, 
                                groups = dataset.ml.group, 
                                cv = splitter,
                                verbose = 1,
                                n_jobs = 1)

#Append these for later use
dataset['prediction'] = predictions

In [ ]:
acc = accuracy_score(dataset.ml.labels, predictions)
f1 = f1_score(dataset.ml.labels, predictions)
pr = precision_score(dataset.ml.labels, predictions)
re = recall_score(dataset.ml.labels, predictions)
print("Acc", acc, "F1", f1, 'precision', pr, 'recall', re)

## Unsupervised learning

In [ ]:
#%%################
## Dim reduction ##
###################

embedding = compute_umap_embedding(dataset, dataset.features.active, N_rows = N_UMAP_ROWS)
dataset[['embedding_0', 'embedding_1']] = embedding

In [ ]:
fig, ax = plot_embedding(dataset, color_col = 'prediction') 

## Post processing

Now we have our model we can make a video of its predictions. Provide the column names whose state we're going to overlay on the video, along with the directory to output the videos.

NOTE: need to have provided 'video' column in the metadata to make movies.

In [ ]:
dataset.io.save_movie(['label', 'prediction'], '.')